In [ ]:
import os
import os.path as op
import mne
import numpy as np
from scipy import stats as stats
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.stats.anova import AnovaRM
import seaborn as sns

from code.analysis.megcoherence_utils import load_data_roi_analysis, load_data_cluster

%matplotlib inline
# making sure text will be text on the output svg
plt.rcParams['svg.fonttype'] = 'none'

In [ ]:
# Loading data
# Anatomical ROI data
roi_data_dir = op.join('/imaging', 'davis', 'users', 'ma09', 'Projects', 'AVSpeechMEG',
                        'data', 'derivatives', 'megcoherence', 'group')
anat_roi_fname = 'megcoherence_anatomical_roi_data.csv'
if not op.exists(op.join(roi_data_dir, anat_roi_fname)):
    load_data_roi_analysis(
        ['pcoh-aud-AV-true', 'pcoh-aud-AV-perm',
         'pcoh-aud-AV-true-matchVO', 'pcoh-aud-AV-perm-matchVO',
         'pcoh-aud-AV20-true', 'pcoh-aud-AV20-perm',
         'pcoh-aud-AV70-true', 'pcoh-aud-AV70-perm',
         'pcoh-vis-AV-true', 'pcoh-vis-AV-perm',
         'pcoh-vis-AV-true-matchVO', 'pcoh-vis-AV-perm-matchVO',
         'pcoh-vis-AV20-true', 'pcoh-vis-AV20-perm',
         'pcoh-vis-AV70-true', 'pcoh-vis-AV70-perm',
         'pcoh-aud-AO-true', 'pcoh-aud-AO-perm',
         'pcoh-aud-AO20-true', 'pcoh-aud-AO20-perm',
         'pcoh-aud-AO70-true', 'pcoh-aud-AO70-perm',
         'pcoh-vis-AO-true', 'pcoh-vis-AO-perm',
         'pcoh-vis-AO-true-matchVO', 'pcoh-vis-AO-perm-matchVO',
         'pcoh-aud-VO-true', 'pcoh-aud-VO-perm',
         'pcoh-vis-VO-true', 'pcoh-vis-VO-perm',
         'pcoh-aud-Ahigh-true', 'pcoh-aud-Ahigh-perm',
         'pcoh-aud-Alow-true', 'pcoh-aud-Alow-perm',
         'pcoh-vis-Ahigh-true', 'pcoh-vis-Ahigh-perm',
         'pcoh-vis-Alow-true', 'pcoh-vis-Alow-perm'],
        ['STG', 'Occ_cortex'], save=True)
df = pd.read_csv(op.join(roi_data_dir, anat_roi_fname))

# Functional ROI data
func_roi_fname = 'megcoherence_functional_roi_data.csv'
if not op.exists(op.join(roi_data_dir, func_roi_fname)):
    load_data_cluster(
        ['pcoh-aud-AV-true', 'pcoh-aud-AV-perm',
         'pcoh-aud-AO-true', 'pcoh-aud-AO-perm',
         'pcoh-vis-AV-true', 'pcoh-vis-AV-perm',
         'pcoh-vis-VO-true', 'pcoh-vis-VO-perm'],
        ['aud_AV>AO', 'aud_AV>AO',
         'aud_AV>AO', 'aud_AV>AO',
         'vis_VO>AV', 'vis_VO>AV',
         'vis_VO>AV', 'vis_VO>AV'], save=True)
df_func = pd.read_csv(op.join(roi_data_dir, func_roi_fname))

# Behavioural data
behav_data_dir = op.join('/imaging', 'davis', 'users', 'ma09', 'Projects', 'AVSpeechMEG',
                        'data', 'derivatives', 'behavioural', 'group')
df_name = 'word_report.csv'
df_wr = pd.read_csv(op.join(behav_data_dir, df_name))

# Set summary plot type, can be box or point
summary_plot_type = 'point'

In [ ]:
# Function to plot data and statistical results for anova
def my_plot_anova(df, within, dep='coh_mean', summary='point', 
                  plot_anova=True, plot_simple=True, one_tail=True, save=False):

    # Compute repeated measures ANOVA
    aovrm = AnovaRM(df, dep, 'subID', within=within)
    res = aovrm.fit()
    print(res)

    # Plot the data
    sns.set_theme(style="ticks")
    f, ax = plt.subplots(figsize=(7, 5))
    if summary == 'box':
        # Add in points to show each observation
        sns.stripplot(x=within[0], y=dep, hue=within[1],
                  data=df, color=".3", linewidth=0, dodge=True)
        # plot data summary using boxplot
        bplot = sns.boxplot(x=within[0], y=dep, hue=within[1],
                            data=df, whis=[0, 100], width=.6, 
                            palette='colorblind')
    elif summary == 'point':
        # Add in points to show each observation
        sns.stripplot(x=within[0], y=dep, hue=within[1],
                  data=df, color=".3", linewidth=0, dodge=True,
                  alpha=.25)
        # plot data summary using pointplot
        pplot = sns.pointplot(x=within[0], y=dep, hue=within[1],
                              data=df, palette='colorblind', dodge=.4, 
                              ci=68, join=False, capsize=.05, markers='d',
                              errwidth=1.5)
    else:
        raise ValueError()
    # Tweak the visual presentation
    sns.despine(trim=True, bottom=True)
    plt.show()
    if save:
        f.savefig('fig_anova.svg')

    # Plot the simple effects
    if plot_simple:
        # Compute simple effects of second factor within the levels of the first factor
        levels0 =sorted(df[within[0]].unique())
        levels1 =sorted(df[within[1]].unique())
        cond1 = df.loc[df[within[0]].isin([levels0[0]]) &
                       df[within[1]].isin([levels1[1]])]
        cond2 = df.loc[df[within[0]].isin([levels0[0]]) &
                       df[within[1]].isin([levels1[0]])]
        cond3 = df.loc[df[within[0]].isin([levels0[1]]) &
                       df[within[1]].isin([levels1[1]])]
        cond4 = df.loc[df[within[0]].isin([levels0[1]]) &
                       df[within[1]].isin([levels1[0]])]
        data1_2 = cond1[dep].to_numpy()-cond2[dep].to_numpy()
        data3_4 = cond3[dep].to_numpy()-cond4[dep].to_numpy()
        st1_2, pval1_2 = stats.ttest_1samp(data1_2, 0)
        st3_4, pval3_4 = stats.ttest_1samp(data3_4, 0)
        # Correcting p-value to one-tailed
        if one_tail:
            ptail = 1
            pval1_2 /= 2
            pval3_4 /= 2
        else:
            ptail = 0   
        # Compute interaction
        interaction = (cond1[dep].to_numpy()-cond2[dep].to_numpy()) - \
            (cond3[dep].to_numpy()-cond4[dep].to_numpy())
        st_int, pval_int = stats.ttest_1samp(interaction, 0)
        text = ''.join(['Simple effect of {} within \'{}\':\n'.format(within[1], levels0[0]),
                        '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data1_2), stats.sem(data1_2)),
                        '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(cond1)-1, 
                                                                                  st1_2, ptail, pval1_2),
                        'Simple effect of {} within \'{}\':\n'.format(within[1], levels0[1]),
                        '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data3_4), stats.sem(data3_4)),
                        '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(cond1)-1, 
                                                                                  st3_4, ptail, pval3_4),
                        'Interaction of {} and {}:\n'.format(within[0], within[1]),
                        '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(cond1)-1, 
                                                                                  st_int, 0, pval_int)])
        print(text)

        sns.set_theme(style="ticks")
        f, ax = plt.subplots(figsize=(7, 5))
        y = np.concatenate([cond1[dep].to_numpy()-cond2[dep].to_numpy(),
                            cond3[dep].to_numpy()-cond4[dep].to_numpy()])
        x = np.ones_like(y)
        x[-len(cond1):] = 2
        if summary == 'box':
            # Add in points to show each observation
            sns.stripplot(x=x, y=y, color=".3", linewidth=0, dodge=True,
                         jitter=.05).set_title('Simple effects')
            # plot data summary using boxplot
            bplot = sns.boxplot(x=x, y=y, whis=[0, 100], width=.6, 
                                palette='colorblind')
        elif summary == 'point':
            # Add in points to show each observation        
            sns.stripplot(x=x, y=y, color=".3", linewidth=0, dodge=True,
                      jitter=.05, alpha=.25).set_title('Simple effects')
            # plot data summary using pointplot
            pplot = sns.pointplot(x=x, y=y, palette='colorblind', dodge=.4, 
                                  ci=68, join=False, capsize=.05, markers='d',
                                  errwidth=1.5)
        else:
            raise ValueError()

        # Tweak the visual presentation
        sns.despine(trim=True, bottom=True)
        ax.set(xlabel=within[0])
        ax.set(ylabel='Rel. coherence {}-{}'.format(levels1[1],levels1[0]))
        plt.xticks((0, 1), [levels0[0], levels0[1]])
        plt.show()
        if save:
            f.savefig('fig_simple_effects.svg')

In [ ]:
def my_plot_ttest(df, within, dep='coh_mean', summary='point', 
                  one_tail=True, save=False):
    
    # compute one-sample ttest
    levels =sorted(df[within[0]].unique())
    cond1 = df.loc[df[within[0]].isin([levels[0]])]
    cond2 = df.loc[df[within[0]].isin([levels[1]])]
    data1_2 = cond1[dep].to_numpy()-cond2[dep].to_numpy()
    st1_2, pval1_2 = stats.ttest_1samp(data1_2, 0)
    # Correcting p-value to one-tailed
    if one_tail:
        ptail = 1
        pval1_2 /= 2
    else:
        ptail = 0   
    text = ''.join(['Effect of {} :\n'.format(within[0]),
                    '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data1_2), stats.sem(data1_2)),
                    '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(cond1)-1, 
                                                                              st1_2, ptail, pval1_2)])
    print(text)

    sns.set_theme(style="ticks")
    f, ax = plt.subplots(figsize=(7, 5))
    y = np.concatenate([cond1[dep].to_numpy(),cond2[dep].to_numpy()])
    x = np.ones_like(y)
    x[-len(cond1):] = 2
    if summary == 'box':
        # Add in points to show each observation
        sns.stripplot(x=x, y=y, color=".3", linewidth=0, dodge=True,
                     jitter=.05).set_title('Simple effects')
        # plot data summary using boxplot
        bplot = sns.boxplot(x=x, y=y, whis=[0, 100], width=.6, 
                            palette='colorblind')
    elif summary == 'point':
        # Add in points to show each observation        
        sns.stripplot(x=x, y=y, color=".3", linewidth=0, dodge=True,
                  jitter=.05, alpha=.25).set_title('Simple effects')
        # plot data summary using pointplot
        pplot = sns.pointplot(x=x, y=y, palette='colorblind', dodge=.4, 
                              ci=68, join=False, capsize=.05, markers='d',
                              errwidth=1.5)
    else:
        raise ValueError()

    # Tweak the visual presentation
    sns.despine(trim=True, bottom=True)
    ax.set(xlabel=within[0])
    plt.xticks((0, 1), [levels[0], levels[1]])
    ax.set(ylabel='{}'.format(dep))
    plt.show()
    if save:
        f.savefig('fig_simple_effects.svg')

# Occipital cortex

**Effect of coherence modality within AV conditions**

All the below results are based on _partial coherence_ values averaged within the respecive ROIs

In [ ]:
df_occ = df[df['roi'] == 'Occ_cortex'].copy()
my_plot_anova(df_occ[df_occ['stim_modality'].isin(['AV']) &
                     df_occ['acc_clarity'].isin(['none']) & 
                     df_occ['match_sample'].isin(['none'])],
              ['coh_modality', 'permutation'], summary=summary_plot_type)

**Occipital cortex - Effect of stimulus modality (AO vs AV) within auditory coherence**

All the below results are based on _partial coherence_ values averaged within the respecive ROI

In [ ]:
my_plot_anova(df_occ[df_occ['coh_modality'].isin(['aud']) &
                     df_occ['stim_modality'].isin(['AV', 'AO']) &
                     df_occ['acc_clarity'].isin(['none']) & 
                     df_occ['match_sample'].isin(['none'])],
              ['stim_modality', 'permutation'], summary=summary_plot_type)

**Occipital cortex - Effect of stimulus modality (VO vs AV) within visual coherence**

All the below results are based on _partial coherence_ values averaged within the respecive ROI

In [ ]:
my_plot_anova(df_occ[df_occ['coh_modality'].isin(['vis']) &
                     df_occ['stim_modality'].isin(['AV', 'VO']) &
                     df_occ['acc_clarity'].isin(['none'])& 
                     df_occ['match_sample'].isin(['none'])],
              ['stim_modality', 'permutation'], summary=summary_plot_type)

**Occipital cortex - Effect of stimulus modality (VO vs AV) within visual coherence (number of trials in AV matched to VO)**

All the below results are based on _partial coherence_ values averaged within the respecive ROI

In [ ]:
my_plot_anova(
    df_occ[df_occ['coh_modality'].isin(['vis']) &  # only visual coherence
           df_occ['acc_clarity'].isin(['none']) &  # mean over acoustic clarity
           ((df_occ['stim_modality'].isin(['AV']) & df_occ['match_sample'].isin(['matchVO'])) |
             df_occ['stim_modality'].isin(['VO']))], # AV matched to VO, and VO
              ['stim_modality', 'permutation'], summary=summary_plot_type)

**Occipital cortex - Effect of stimulus modality (AO vs AV) and acoustic clarity (high vs low) within auditory coherence**

All the below results are based on _partial coherence_ values averaged within the respecive ROI

In [ ]:
df_temp = df_occ[df_occ['coh_modality'].isin(['aud']) &
                 df_occ['stim_modality'].isin(['AV', 'AO']) &
                 df_occ['acc_clarity'].isin(['high', 'low'])]
df_clarity_true = df_temp[df_temp['permutation'].isin(['true'])].sort_values(
    by=['subID', 'stim_modality', 'acc_clarity'])
df_clarity_perm = df_temp[df_temp['permutation'].isin(['perm'])].sort_values(
    by=['subID', 'stim_modality', 'acc_clarity'])
df_clarity = df_clarity_true.copy().drop('permutation', axis=1)
df_clarity[['coh_mean', 'coh_max']] = \
    df_clarity_true[['coh_mean', 'coh_max']].to_numpy() - \
    df_clarity_perm[['coh_mean', 'coh_max']].to_numpy()
my_plot_anova(df_clarity, ['stim_modality', 'acc_clarity'], 
              summary=summary_plot_type, one_tail=False)

**Occipital cortex - Auditory partial coherence in VO**

All the below results are based on partial coherence values averaged within the respecive ROI

In [ ]:
my_plot_anova(df_occ[df_occ['coh_modality'].isin(['aud']) &
                     df_occ['stim_modality'].isin(['AV', 'VO']) &
                     df_occ['acc_clarity'].isin(['none']) & 
                     df_occ['match_sample'].isin(['none'])],
              ['stim_modality', 'permutation'], summary=summary_plot_type)

**Occipital cortex - Visual partial coherence in AO**

All the below results are based on partial coherence values averaged within the respecive ROI

In [ ]:
my_plot_anova(df_occ[df_occ['coh_modality'].isin(['vis']) &
                     df_occ['stim_modality'].isin(['AV', 'AO']) &
                     df_occ['acc_clarity'].isin(['none']) & 
                     df_occ['match_sample'].isin(['none'])],
              ['stim_modality', 'permutation'], summary=summary_plot_type)

**Occipital cortex - Effect of stimulus modality with matched intelligibility (AV_low vs AO_high) within auditory coherence**

All the below results are based on _partial coherence_ values averaged within the respecive ROI

In [ ]:
my_plot_anova(
    df_occ[df_occ['coh_modality'].isin(['aud']) &  # only auditory coherence
           ((df_occ['stim_modality'].isin(['AV']) & df_occ['acc_clarity'].isin(['low'])) |
             df_occ['stim_modality'].isin(['AO']) & df_occ['acc_clarity'].isin(['high']))],
              ['stim_modality', 'permutation'], summary=summary_plot_type)

# STG

**Effect of coherence modality within AV conditions**

All the below results are based on _partial coherence_ values averaged within the respecive ROIs

In [ ]:
df_stg = df[df['roi'] == 'STG'].copy()
my_plot_anova(df_stg[df_stg['stim_modality'].isin(['AV']) &
                     df_stg['acc_clarity'].isin(['none']) & 
                     df_stg['match_sample'].isin(['none'])],
              ['coh_modality', 'permutation'], summary=summary_plot_type)

**STG - Effect of stimulus modality (AV vs AO) within auditory coherence**

All the below results are based on _partial coherence_ values averaged within the respecive ROI

In [ ]:
my_plot_anova(df_stg[df_stg['coh_modality'].isin(['aud']) &
                     df_stg['stim_modality'].isin(['AV', 'AO']) &
                     df_stg['acc_clarity'].isin(['none']) &
                     df_stg['match_sample'].isin(['none'])],
              ['stim_modality', 'permutation'], summary=summary_plot_type)

**STG - Effect of stimulus modality (AV vs VO) within visual coherence**

All the below results are based on _partial coherence_ values averaged within the respecive ROI

In [ ]:
my_plot_anova(df_stg[df_stg['coh_modality'].isin(['vis']) &
                     df_stg['stim_modality'].isin(['AV', 'VO']) &
                     df_stg['acc_clarity'].isin(['none']) & 
                     df_stg['match_sample'].isin(['none'])],
              ['stim_modality', 'permutation'], summary=summary_plot_type)

**STG - Effect of stimulus modality (AV vs VO) within visual coherence (number of trials in AV matched to VO)**

All the below results are based on _partial coherence_ values averaged within the respecive ROI

In [ ]:
my_plot_anova(
    df_stg[df_stg['coh_modality'].isin(['vis']) &  # only visual coherence
           df_stg['acc_clarity'].isin(['none']) &  # mean over acoustic clarity
           ((df_stg['stim_modality'].isin(['AV']) & df_stg['match_sample'].isin(['matchVO'])) |
             df_stg['stim_modality'].isin(['VO']))], # AV matched to VO, and VO
              ['stim_modality', 'permutation'], summary=summary_plot_type)

**STG - Effect of stimulus modality (VO vs AV) and acoustic clarity (high vs low) within visual coherence**

All the below results are based on _partial coherence_ values averaged within the respecive ROI

In [ ]:
df_temp_AV = df_stg[df_stg['coh_modality'].isin(['vis']) &
                 df_stg['stim_modality'].isin(['AV']) &
                 df_stg['acc_clarity'].isin(['high', 'low'])]
df_temp_VO = df_stg[df_stg['coh_modality'].isin(['vis']) &
                 df_stg['stim_modality'].isin(['VO'])]
# Here I pretend there were high and low clarity levels in VO as well, 
# basically just duplicating the data. This is just because I can reuse the
# plotting function from above, ANOVA results will not be valid!
t1 = df_temp_VO.copy()
t1['acc_clarity'] = 'high'
t2 = df_temp_VO.copy()
t2['acc_clarity'] = 'low'
df_temp_VO = pd.concat([t1, t2], ignore_index=True)
df_temp = pd.concat([df_temp_AV, df_temp_VO], ignore_index=True)
df_clarity_true = df_temp[df_temp['permutation'].isin(['true'])].sort_values(
    by=['subID', 'stim_modality', 'acc_clarity'])
df_clarity_perm = df_temp[df_temp['permutation'].isin(['perm'])].sort_values(
    by=['subID', 'stim_modality', 'acc_clarity'])
df_clarity = df_clarity_true.copy().drop('permutation', axis=1)
df_clarity[['coh_mean', 'coh_max']] = \
    df_clarity_true[['coh_mean', 'coh_max']].to_numpy() - \
    df_clarity_perm[['coh_mean', 'coh_max']].to_numpy()
my_plot_anova(df_clarity, ['stim_modality', 'acc_clarity'], summary=summary_plot_type,
             plot_simple=True)
# Pairwise comparisons between AVhigh and VO, and AVlow and VO
AVhigh = df_clarity.loc[df_clarity['stim_modality'].isin(['AV']) & 
                        df_clarity['acc_clarity'].isin(['high']), 'coh_mean'].to_numpy()
AVlow = df_clarity.loc[df_clarity['stim_modality'].isin(['AV']) & 
                       df_clarity['acc_clarity'].isin(['low']), 'coh_mean'].to_numpy()
VO = df_clarity.loc[df_clarity['stim_modality'].isin(['VO']) & 
                    df_clarity['acc_clarity'].isin(['high']), 'coh_mean'].to_numpy()  # high/low irrelevant here
st1_2, pval1_2 = stats.ttest_1samp(VO-AVhigh, 0)
st3_4, pval3_4 = stats.ttest_1samp(VO-AVlow, 0) 
text = ''.join(['Pairwise comparison between VO and AV_high\n',
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(AVhigh)-1, 
                                                                          st1_2, 0, pval1_2),
                'Pairwise comparison between VO and AV_low\n',
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(AVlow)-1, 
                                                                          st3_4, 0, pval3_4)])
print(text)

**STG - Visual partial coherence in AO**

In [ ]:
my_plot_anova(df_stg[df_stg['coh_modality'].isin(['vis']) &
                     df_stg['stim_modality'].isin(['AV', 'AO']) &
                     df_stg['acc_clarity'].isin(['none']) & 
                     df_stg['match_sample'].isin(['none'])],
              ['stim_modality', 'permutation'], summary=summary_plot_type)

**STG - Auditory partial coherence in VO**

In [ ]:
my_plot_anova(df_stg[df_stg['coh_modality'].isin(['aud']) &
                     df_stg['stim_modality'].isin(['AV', 'VO']) &
                     df_stg['acc_clarity'].isin(['none']) & 
                     df_stg['match_sample'].isin(['none'])],
              ['stim_modality', 'permutation'], summary=summary_plot_type)

**STG - Effect of acoustic clarity (high vs low), auditory coherence**

All the below results are based on _partial coherence_ values averaged within the respecive ROI

In [ ]:
my_plot_anova(df_stg[df_stg['coh_modality'].isin(['aud']) &
                     df_stg['stim_modality'].isin(['A'])],
              ['acc_clarity', 'permutation'], summary=summary_plot_type)

**STG - Effect of acoustic clarity (high vs low), visual coherence**

All the below results are based on _partial coherence_ values averaged within the respecive ROI

In [ ]:
my_plot_anova(df_stg[df_stg['coh_modality'].isin(['vis']) &
                     df_stg['stim_modality'].isin(['A'])],
              ['acc_clarity', 'permutation'], summary=summary_plot_type)

**STG - Effect of stimulus modality with matched intelligibility (AV_low vs AO_high) within auditory coherence**

All the below results are based on _partial coherence_ values averaged within the respecive ROI

In [ ]:
my_plot_anova(
    df_stg[df_stg['coh_modality'].isin(['aud']) &  # only auditory coherence
           ((df_stg['stim_modality'].isin(['AV']) & df_stg['acc_clarity'].isin(['low'])) |
             df_stg['stim_modality'].isin(['AO']) & df_stg['acc_clarity'].isin(['high']))],
              ['stim_modality', 'permutation'], summary=summary_plot_type, save=True)

# Partial coherence (true-permuted) for AV auditory vs visual separately by ROIs

In [ ]:
# Plotting true-permuted AV aud vs vis in one plot
# Make sure the dataframe is properly sorted
df_true = df[df['permutation'].isin(['true'])].copy().sort_values(
    by=['subID', 'coh_modality', 'stim_modality', 'acc_clarity', 'match_sample', 'roi'])
df_perm = df[df['permutation'].isin(['perm'])].copy().sort_values(
    by=['subID', 'coh_modality', 'stim_modality', 'acc_clarity', 'match_sample', 'roi'])
df_tr_p = df_true.copy().drop('permutation', axis=1)
df_tr_p['coh_mean'] = df_true['coh_mean'].to_numpy()-df_perm['coh_mean'].to_numpy()
df_tr_p['coh_max'] = df_true['coh_max'].to_numpy()-df_perm['coh_max'].to_numpy()
data = df_tr_p[df_tr_p['stim_modality'].isin(['AV']) & 
               df_tr_p['acc_clarity'].isin(['none']) &
               df_tr_p['match_sample'].isin(['none'])]
# Compute repeated measures ANOVA
aovrm = AnovaRM(data, 'coh_mean', 'subID', within=['coh_modality', 'roi'])
res = aovrm.fit()
print(res)

# compute one-sample ttests
cond1 = data.loc[data['roi'].isin(['Occ_cortex']) & 
                 data['coh_modality'].isin(['vis'])]
cond2 = data.loc[data['roi'].isin(['Occ_cortex']) & 
                 data['coh_modality'].isin(['aud'])]
data1 = cond1['coh_mean'].to_numpy()
data2 = cond2['coh_mean'].to_numpy()
# vis in AV in Occ
st1, pval1 = stats.ttest_1samp(data1, 0)
# aud in AV in Occ
st2, pval2 = stats.ttest_1samp(data2, 0)
cond3 = data.loc[data['roi'].isin(['STG']) & 
                 data['coh_modality'].isin(['vis'])]
cond4 = data.loc[data['roi'].isin(['STG']) & 
                 data['coh_modality'].isin(['aud'])]
data3 = cond3['coh_mean'].to_numpy()
data4 = cond4['coh_mean'].to_numpy()
# vis in AV in STG
st3, pval3 = stats.ttest_1samp(data3, 0)
# aud in AV in STG
st4, pval4 = stats.ttest_1samp(data4, 0)

# compare effects across ROIs
# vis in AV vs. aud in AV withing Occ
data1_2 = data1-data2
st1_2, pval1_2 = stats.ttest_1samp(data1_2, 0)
# vis in AV vs. aud in AV withing STG
data3_4 = data3-data4
st3_4, pval3_4 = stats.ttest_1samp(data3_4, 0)
# vis in AV in Occ vs. vis in AO in STG
data1_3 = data1-data3
st1_3, pval1_3 = stats.ttest_1samp(data1_3, 0)
# aud in VO in Occ vs. aud in VO in STG
data2_4 = data2-data4
st2_4, pval2_4 = stats.ttest_1samp(data2_4, 0)

# Correcting p-value to one-tailed
one_tail = True
if one_tail:
    ptail = 1
    pval1 /= 2
    pval2 /= 2
    pval3 /= 2
    pval4 /= 2
else:
    ptail = 0   
text = ''.join(['\'{}\' within \'{}\':\n'.format('vis in AV','Occ_cortex'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data1), stats.sem(data1)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(cond1)-1, 
                                                                          st1, ptail, pval1),
                '\'{}\' within \'{}\':\n'.format('aud in AV','Occ_cortex'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data2), stats.sem(data2)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(cond2)-1, 
                                                                          st2, ptail, pval2),
                '\'{}\' within \'{}\':\n'.format('vis in AV','STG'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data3), stats.sem(data3)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(cond3)-1, 
                                                                          st3, ptail, pval3),
                '\'{}\' within \'{}\':\n'.format('aud in AV','STG'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data4), stats.sem(data4)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n\n'.format(len(cond4)-1, 
                                                                          st4, ptail, pval4),
                '\'{}\' vs. \'{}\' within \'{}\':\n'.format('vis in AV', 'aud in AV', 'Occ_cortex'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data1_2), stats.sem(data1_2)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(cond1)-1, 
                                                                          st1_2, 0, pval1_2),
                '\'{}\' vs. \'{}\' within \'{}\':\n'.format('vis in AV', 'aud in AV', 'STG'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data3_4), stats.sem(data3_4)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(cond3)-1, 
                                                                          st3_4, 0, pval3_4),
                '\'{}\' vs. \'{}\':\n'.format('vis in AV in Occ','vis in AV in STG'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data1_3), stats.sem(data1_3)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(cond1)-1, 
                                                                          st1_3, 0, pval1_3),
                '\'{}\' vs. \'{}\':\n'.format('aud in AV in Occ','aud in AV in STG'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data2_4), stats.sem(data2_4)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(cond2)-1, 
                                                                          st2_4, 0, pval2_4)])
print(text)

# Plot the data
sns.set_theme(style="ticks")
f, ax = plt.subplots(figsize=(7, 5))
if summary_plot_type == 'box':
    # Add in points to show each observation
    sns.stripplot(x='roi', y='coh_mean', hue='coh_modality',
                  data=data, color=".3", linewidth=0, dodge=True)
    # plot data summary using boxplot
    bplot = sns.boxplot(x='roi', y='coh_mean', hue='coh_modality',
                        data=data, whis=[0, 100], width=.6, 
                        palette='colorblind')
elif summary_plot_type == 'point':
    # plot data summary using pointplot
    pplot = sns.pointplot(x='roi', y='coh_mean', hue='coh_modality',
                          data=data, palette='colorblind', dodge=.4, 
                          ci=68, join=False, capsize=.05, markers='d',
                          errwidth=1.5) 
# This is a way to plot arbitrary errorbars, not very simple, but works
#     Find the x,y coordinates for each point
#     x_coords = []
#     y_coords = []
#     for point_pair in pplot.collections:
#         for x, y in point_pair.get_offsets():
#             x_coords.append(x)
#             y_coords.append(y)

#     # Calculate the type of error to plot as the error bars
#     # Make sure the order is the same as the points were looped over
#     errors = data.drop('partialized', axis=1).groupby(['coh_modality', 'roi']).sem()['coh_mean']
#     colors = ['steelblue']*2 + ['coral']*2
#     pplot.errorbar(x_coords, y_coords, yerr=errors,
#         ecolor=colors, fmt=' ', zorder=-1)
    
    # Add in points to show each observation
    sns.stripplot(x='roi', y='coh_mean', hue='coh_modality',
                  data=data, color=".3", linewidth=0, dodge=True,
                  alpha=.25)

# Tweak the visual presentation
sns.despine(trim=False, bottom=True)
plt.setp(ax, ylim=(-0.01,0.04))
plt.show()
# f.savefig('fig_AV_cohmod_x_roi.svg')

# Partial coherence (true-permuted), auditory in VO vs visual in AO separately by ROIs

In [ ]:
# Make sure the dataframe is properly sorted
df_true = df[df['permutation'].isin(['true'])].copy().sort_values(
    by=['subID', 'coh_modality', 'stim_modality', 'acc_clarity', 'match_sample', 'roi'])
df_perm = df[df['permutation'].isin(['perm'])].copy().sort_values(
    by=['subID', 'coh_modality', 'stim_modality', 'acc_clarity', 'match_sample', 'roi'])
df_tr_p = df_true.copy().drop('permutation', axis=1)
df_tr_p['coh_mean'] = df_true['coh_mean'].to_numpy()-df_perm['coh_mean'].to_numpy()
df_tr_p['coh_max'] = df_true['coh_max'].to_numpy()-df_perm['coh_max'].to_numpy()
data = df_tr_p[(df_tr_p['stim_modality'].isin(['AO']) & df_tr_p['coh_modality'].isin(['vis']) & df_tr_p['match_sample'].isin(['none'])) | 
               (df_tr_p['stim_modality'].isin(['VO']) & df_tr_p['coh_modality'].isin(['aud']))]

# compute one-sample ttests
cond1 = data.loc[data['roi'].isin(['Occ_cortex']) & 
                 data['coh_modality'].isin(['vis'])]
cond2 = data.loc[data['roi'].isin(['Occ_cortex']) & 
                 data['coh_modality'].isin(['aud'])]
data1 = cond1['coh_mean'].to_numpy()
data2 = cond2['coh_mean'].to_numpy()
# vis in AO in Occ
st1, pval1 = stats.ttest_1samp(data1, 0)
# aud in VO in Occ
st2, pval2 = stats.ttest_1samp(data2, 0)
cond3 = data.loc[data['roi'].isin(['STG']) & 
                 data['coh_modality'].isin(['vis'])]
cond4 = data.loc[data['roi'].isin(['STG']) & 
                 data['coh_modality'].isin(['aud'])]
data3 = cond3['coh_mean'].to_numpy()
data4 = cond4['coh_mean'].to_numpy()
# vis in AO in STG
st3, pval3 = stats.ttest_1samp(data3, 0)
# aud in VO in STG
st4, pval4 = stats.ttest_1samp(data4, 0)

# compare effects across ROIs
# vis in AO in Occ vs. vis in AO in STG
data1_3 = data1-data3
st1_3, pval1_3 = stats.ttest_1samp(data1_3, 0)
# aud in VO in Occ vs. aud in VO in STG
data2_4 = data2-data4
st2_4, pval2_4 = stats.ttest_1samp(data2_4, 0)

# Correcting p-value to one-tailed
one_tail = True
if one_tail:
    ptail = 1
    pval1 /= 2
    pval2 /= 2
    pval3 /= 2
    pval4 /= 2
else:
    ptail = 0   
text = ''.join(['\'{}\' within \'{}\':\n'.format('vis in AO','Occ_cortex'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data1), stats.sem(data1)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(cond1)-1, 
                                                                          st1, ptail, pval1),
                '\'{}\' within \'{}\':\n'.format('aud in VO','Occ_cortex'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data2), stats.sem(data2)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(cond2)-1, 
                                                                          st2, ptail, pval2),
                '\'{}\' within \'{}\':\n'.format('vis in AO','STG'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data3), stats.sem(data3)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(cond3)-1, 
                                                                          st3, ptail, pval3),
                '\'{}\' within \'{}\':\n'.format('aud in VO','STG'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data4), stats.sem(data4)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n\n'.format(len(cond4)-1, 
                                                                          st4, ptail, pval4),
                '\'{}\' vs. \'{}\':\n'.format('vis in AO in Occ','vis in AO in STG'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data1_3), stats.sem(data1_3)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(cond1)-1, 
                                                                          st1_3, 0, pval1_3),
                '\'{}\' vs. \'{}\':\n'.format('aud in VO in Occ','aud in VO in STG'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data2_4), stats.sem(data2_4)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(cond2)-1, 
                                                                          st2_4, 0, pval2_4)])
print(text)

sns.set_theme(style="ticks")
f, ax = plt.subplots(figsize=(7, 5))
if summary_plot_type == 'box':
    # Add in points to show each observation
    sns.stripplot(x='roi', y='coh_mean', hue='coh_modality',
                  data=data, color=".3", linewidth=0, dodge=True)
    # plot data summary using boxplot
    bplot = sns.boxplot(x='roi', y='coh_mean', hue='coh_modality',
                        data=data, whis=[0, 100], width=.6, 
                        palette='colorblind')
elif summary_plot_type == 'point':
    # plot data summary using pointplot
    pplot = sns.pointplot(x='roi', y='coh_mean', hue='coh_modality',
                          data=data, palette='colorblind', dodge=.4, 
                          ci=68, join=False, capsize=.05, markers='d',
                          errwidth=1.5)
    # Add in points to show each observation
    sns.stripplot(x='roi', y='coh_mean', hue='coh_modality',
                  data=data, color=".3", linewidth=0, dodge=True,
                  alpha=.25)

# Tweak the visual presentation
sns.despine(trim=False, bottom=True)
plt.setp(ax, ylim=(-0.01,0.04))
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', ncol=1)
plt.show()

# f.savefig('fig_AO_VO_cohmod_x_roi.svg')

# Partial coherence (true-permuted), auditory in VO vs visual in AO (sample size matched to VO) separately by ROIs

In [ ]:
# Make sure the dataframe is properly sorted
df_true = df[df['permutation'].isin(['true'])].copy().sort_values(
    by=['subID', 'coh_modality', 'stim_modality', 'acc_clarity', 'match_sample', 'roi'])
df_perm = df[df['permutation'].isin(['perm'])].copy().sort_values(
    by=['subID', 'coh_modality', 'stim_modality', 'acc_clarity', 'match_sample', 'roi'])
df_tr_p = df_true.copy().drop('permutation', axis=1)
df_tr_p['coh_mean'] = df_true['coh_mean'].to_numpy()-df_perm['coh_mean'].to_numpy()
df_tr_p['coh_max'] = df_true['coh_max'].to_numpy()-df_perm['coh_max'].to_numpy()
data = df_tr_p[(df_tr_p['stim_modality'].isin(['AO']) & df_tr_p['coh_modality'].isin(['vis']) & df_tr_p['match_sample'].isin(['matchVO'])) | 
               (df_tr_p['stim_modality'].isin(['VO']) & df_tr_p['coh_modality'].isin(['aud']))]

# Compute repeated measures ANOVA
aovrm = AnovaRM(data, 'coh_mean', 'subID', within=['coh_modality', 'roi'])
res = aovrm.fit()
print(res)

# compute one-sample ttests
cond1 = data.loc[data['roi'].isin(['Occ_cortex']) & 
                 data['coh_modality'].isin(['vis'])]
cond2 = data.loc[data['roi'].isin(['Occ_cortex']) & 
                 data['coh_modality'].isin(['aud'])]
data1 = cond1['coh_mean'].to_numpy()
data2 = cond2['coh_mean'].to_numpy()
# vis in AO in Occ
st1, pval1 = stats.ttest_1samp(data1, 0)
# aud in VO in Occ
st2, pval2 = stats.ttest_1samp(data2, 0)
cond3 = data.loc[data['roi'].isin(['STG']) & 
                 data['coh_modality'].isin(['vis'])]
cond4 = data.loc[data['roi'].isin(['STG']) & 
                 data['coh_modality'].isin(['aud'])]
data3 = cond3['coh_mean'].to_numpy()
data4 = cond4['coh_mean'].to_numpy()
# vis in AO in STG
st3, pval3 = stats.ttest_1samp(data3, 0)
# aud in VO in STG
st4, pval4 = stats.ttest_1samp(data4, 0)

# compare effects across ROIs
# vis in AO in Occ vs. vis in AO in STG
data1_3 = data1-data3
st1_3, pval1_3 = stats.ttest_1samp(data1_3, 0)
# aud in VO in Occ vs. aud in VO in STG
data2_4 = data2-data4
st2_4, pval2_4 = stats.ttest_1samp(data2_4, 0)
# vis in AO in Occ vs. aud in VO in Occ
data1_2 = data1-data2
st1_2, pval1_2 = stats.ttest_1samp(data1_2, 0)
# vis in AO in STG vs. aud in VO in STG
data3_4 = data3-data4
st3_4, pval3_4 = stats.ttest_1samp(data3_4, 0)

# Correcting p-value to one-tailed
one_tail = True
if one_tail:
    ptail = 1
    pval1 /= 2
    pval2 /= 2
    pval3 /= 2
    pval4 /= 2
else:
    ptail = 0   
text = ''.join(['\'{}\' within \'{}\':\n'.format('vis in AO','Occ_cortex'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data1), stats.sem(data1)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(cond1)-1, 
                                                                          st1, ptail, pval1),
                '\'{}\' within \'{}\':\n'.format('aud in VO','Occ_cortex'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data2), stats.sem(data2)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(cond2)-1, 
                                                                          st2, ptail, pval2),
                '\'{}\' within \'{}\':\n'.format('vis in AO','STG'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data3), stats.sem(data3)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(cond3)-1, 
                                                                          st3, ptail, pval3),
                '\'{}\' within \'{}\':\n'.format('aud in VO','STG'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data4), stats.sem(data4)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n\n'.format(len(cond4)-1, 
                                                                          st4, ptail, pval4),
                '\'{}\' vs. \'{}\':\n'.format('vis in AO in Occ','vis in AO in STG'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data1_3), stats.sem(data1_3)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(cond1)-1, 
                                                                          st1_3, 0, pval1_3),
                '\'{}\' vs. \'{}\':\n'.format('aud in VO in Occ','aud in VO in STG'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data2_4), stats.sem(data2_4)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(cond2)-1, 
                                                                          st2_4, 0, pval2_4),
                '\'{}\' vs. \'{}\':\n'.format('vis in AO in Occ','aud in VO in Occ'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data1_2), stats.sem(data1_2)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(cond1)-1, 
                                                                          st1_2, 0, pval1_2),
                '\'{}\' vs. \'{}\':\n'.format('vis in AO in STG','aud in VO in STG'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data3_4), stats.sem(data3_4)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(cond3)-1, 
                                                                          st3_4, 0, pval3_4)])
print(text)

sns.set_theme(style="ticks")
f, ax = plt.subplots(figsize=(7, 5))
if summary_plot_type == 'box':
    # Add in points to show each observation
    sns.stripplot(x='roi', y='coh_mean', hue='coh_modality',
                  data=data, color=".3", linewidth=0, dodge=True)
    # plot data summary using boxplot
    bplot = sns.boxplot(x='roi', y='coh_mean', hue='coh_modality',
                        data=data, whis=[0, 100], width=.6, 
                        palette='colorblind')
elif summary_plot_type == 'point':
    # plot data summary using pointplot
    pplot = sns.pointplot(x='roi', y='coh_mean', hue='coh_modality',
                          data=data, palette='colorblind', dodge=.4, 
                          ci=68, join=False, capsize=.05, markers='d',
                          errwidth=1.5)
    # Add in points to show each observation
    sns.stripplot(x='roi', y='coh_mean', hue='coh_modality',
                  data=data, color=".3", linewidth=0, dodge=True,
                  alpha=.25)

# Tweak the visual presentation
sns.despine(trim=True, bottom=True)
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', ncol=1)
plt.show()

# f.savefig('fig_AO_VO_cohmod_x_roi.svg')

# Partial coherence (true-permuted, sample size matched to VO)
# 2 (auditory vs. visual) x 2 (cross-modal vs. unimodal) x 2 (STG vs Occ) ANOVA

In [ ]:
# Make sure the dataframe is properly sorted
df_true = df[df['permutation'].isin(['true'])].copy().sort_values(
    by=['subID', 'coh_modality', 'stim_modality', 'acc_clarity', 'match_sample', 'roi'])
df_perm = df[df['permutation'].isin(['perm'])].copy().sort_values(
    by=['subID', 'coh_modality', 'stim_modality', 'acc_clarity', 'match_sample', 'roi'])
df_tr_p = df_true.copy().drop('permutation', axis=1)
df_tr_p['coh_mean'] = df_true['coh_mean'].to_numpy()-df_perm['coh_mean'].to_numpy()
df_tr_p['coh_max'] = df_true['coh_max'].to_numpy()-df_perm['coh_max'].to_numpy()
data = df_tr_p[(df_tr_p['stim_modality'].isin(['AO']) & df_tr_p['coh_modality'].isin(['vis']) & df_tr_p['match_sample'].isin(['matchVO'])) | 
               (df_tr_p['stim_modality'].isin(['VO']) & df_tr_p['coh_modality'].isin(['aud'])) | 
               (df_tr_p['stim_modality'].isin(['AV']) & df_tr_p['match_sample'].isin(['matchVO']))]
temp = ['bimodal' if s == 'AV' else 'unimodal' for s  in data['stim_modality']]
data.insert(2, 'modality', temp, True)

# Compute repeated measures ANOVA
print('2 x 2 x 2 anova')
aovrm = AnovaRM(data, 'coh_mean', 'subID', within=['coh_modality', 'modality', 'roi'])
res = aovrm.fit()
print(res)
print('2 x 2 anova for STG')
aovrm = AnovaRM(data[data['roi'].isin(['STG'])], 'coh_mean', 'subID', within=['coh_modality', 'modality'])
res = aovrm.fit()
print(res)
print('2 x 2 anova for Occipital cortex')
aovrm = AnovaRM(data[data['roi'].isin(['Occ_cortex'])], 'coh_mean', 'subID', within=['coh_modality', 'modality'])
res = aovrm.fit()
print(res)

# compute pairwise comparisons
d_vis_uni_occ = data.loc[data['roi'].isin(['Occ_cortex']) & 
                         data['coh_modality'].isin(['vis']) & 
                         data['modality'].isin(['unimodal']),'coh_mean'].to_numpy()
d_aud_uni_occ = data.loc[data['roi'].isin(['Occ_cortex']) & 
                         data['coh_modality'].isin(['aud']) & 
                         data['modality'].isin(['unimodal']),'coh_mean'].to_numpy()
d_vis_uni_stg = data.loc[data['roi'].isin(['STG']) & 
                         data['coh_modality'].isin(['vis']) & 
                         data['modality'].isin(['unimodal']),'coh_mean'].to_numpy()
d_aud_uni_stg = data.loc[data['roi'].isin(['STG']) & 
                         data['coh_modality'].isin(['aud']) & 
                         data['modality'].isin(['unimodal']),'coh_mean'].to_numpy()
d_vis_bi_occ = data.loc[data['roi'].isin(['Occ_cortex']) & 
                        data['coh_modality'].isin(['vis']) & 
                        data['modality'].isin(['bimodal']),'coh_mean'].to_numpy()
d_aud_bi_occ = data.loc[data['roi'].isin(['Occ_cortex']) & 
                        data['coh_modality'].isin(['aud']) & 
                        data['modality'].isin(['bimodal']),'coh_mean'].to_numpy()
d_vis_bi_stg = data.loc[data['roi'].isin(['STG']) & 
                        data['coh_modality'].isin(['vis']) & 
                        data['modality'].isin(['bimodal']),'coh_mean'].to_numpy()
d_aud_bi_stg = data.loc[data['roi'].isin(['STG']) & 
                        data['coh_modality'].isin(['aud']) & 
                        data['modality'].isin(['bimodal']),'coh_mean'].to_numpy()

# compare effects coh_modalities within ROIs
data1_2 = d_vis_uni_occ-d_aud_uni_occ
st1_2, pval1_2 = stats.ttest_1samp(data1_2, 0)
data3_4 = d_vis_uni_stg-d_aud_uni_stg
st1_2, pval1_2 = stats.ttest_1samp(data1_2, 0)

# compare effects across modalities within ROIs
# uni vs bimodal in auditory coherence in Occ
data2_6 = d_aud_uni_occ-d_aud_bi_occ
st2_6, pval2_6 = stats.ttest_1samp(data2_6, 0)
# uni vs bimodal in auditory coherence in STG
data4_8 = d_aud_uni_stg-d_aud_bi_stg
st4_8, pval4_8 = stats.ttest_1samp(data4_8, 0)
# uni vs bimodal in visual coherence in Occ
data1_5 = d_vis_uni_occ-d_vis_bi_occ
st1_5, pval1_5 = stats.ttest_1samp(data1_5, 0)
# uni vs bimodal in visual coherence in STG
data3_7 = d_vis_uni_stg-d_vis_bi_stg
st3_7, pval3_7 = stats.ttest_1samp(data3_7, 0)

# Printing results as formatted text
text = ''.join(['\'{}\' vs. \'{}\':\n'.format('vis unimodal in STG','aud unimodal in STG'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data3_4), stats.sem(data3_4)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(data3_4)-1, 
                                                                          st3_4, 0, pval3_4),
                '\'{}\' vs. \'{}\':\n'.format('vis unimodal in Occ','aud unimodal in Occ'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data1_2), stats.sem(data1_2)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(data1_2)-1, 
                                                                          st1_2, 0, pval1_2),
                
                '\'{}\' vs. \'{}\':\n'.format('aud unimodal in STG','aud bimodal in STG'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data4_8), stats.sem(data4_8)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(data4_8)-1, 
                                                                          st4_8, 0, pval4_8),
                '\'{}\' vs. \'{}\':\n'.format('vis unimodal in STG','vis bimodal in STG'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data3_7), stats.sem(data3_7)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(data3_7)-1, 
                                                                          st3_7, 0, pval3_7),
               '\'{}\' vs. \'{}\':\n'.format('aud unimodal in Occ','aud bimodal in Occ'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data2_6), stats.sem(data2_6)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(data2_6)-1, 
                                                                          st2_6, 0, pval2_6),
               '\'{}\' vs. \'{}\':\n'.format('vis unimodal in Occ','vis bimodal in Occ'),
                '\tMean +/- SEM: {:.4f} +/- {:.4f}\n'.format(np.mean(data1_5), stats.sem(data1_5)),
                '\tOne-sample ttest: t({})={:.3f}, p({})={:.4f}\n'.format(len(data1_5)-1, 
                                                                          st1_5, 0, pval1_5),])
print(text)

sns.set_theme(style="ticks")
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
# plot data summary using pointplot
d1 = data.loc[data['modality'] == 'bimodal', :]
d2 = data.loc[data['modality'] == 'unimodal', :]
pplot = sns.pointplot(x='roi', y='coh_mean', hue='coh_modality',
                      data=d1, palette='colorblind', dodge=.4, 
                      ci=68, join=False, capsize=.05, markers='d',
                      errwidth=1.5, ax=ax1)
# Add in points to show each observation
sns.stripplot(x='roi', y='coh_mean', hue='coh_modality',
              data=d1, color=".3", linewidth=0, dodge=True,
              alpha=.25, ax=ax1)
pplot = sns.pointplot(x='roi', y='coh_mean', hue='coh_modality',
                    data=d2, palette='colorblind', dodge=.4, 
                    ci=68, join=False, capsize=.05, markers='d',
                    errwidth=1.5, ax=ax2)
# Add in points to show each observation
sns.stripplot(x='roi', y='coh_mean', hue='coh_modality',
              data=d2, color=".3", linewidth=0, dodge=True,
              alpha=.25, ax=ax2)

# Tweak the visual presentation
sns.despine(trim=False, bottom=True)
ax1.set_title('bimodal')
ax2.set_title('unimodal')
plt.setp((ax1, ax2), ylim=(-0.01,0.03))
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', ncol=1)
plt.show()

# f.savefig('fig_cohmod_x_modality_x_roi.svg')

# Partial auditory coherence (true-permuted) for AV vs AO separately by ROIs

In [ ]:
# Make sure the dataframe is properly sorted
df_true = df[df['permutation'].isin(['true'])].copy().sort_values(
    by=['subID', 'coh_modality', 'stim_modality', 'acc_clarity', 'match_sample', 'roi'])
df_perm = df[df['permutation'].isin(['perm'])].copy().sort_values(
    by=['subID', 'coh_modality', 'stim_modality', 'acc_clarity', 'match_sample', 'roi'])
df_tr_p = df_true.copy().drop('permutation', axis=1)
df_tr_p['coh_mean'] = df_true['coh_mean'].to_numpy()-df_perm['coh_mean'].to_numpy()
df_tr_p['coh_max'] = df_true['coh_max'].to_numpy()-df_perm['coh_max'].to_numpy()
data = df_tr_p[df_tr_p['coh_modality'].isin(['aud']) & 
               df_tr_p['stim_modality'].isin(['AV', 'AO']) & 
               df_tr_p['acc_clarity'].isin(['none']) &
               df_tr_p['match_sample'].isin(['none'])]

# Anova
aovrm = AnovaRM(data, 'coh_mean', 'subID', within=['stim_modality', 'roi'])
res = aovrm.fit()
print(res)

# Plot the data
sns.set_theme(style="ticks")
f, ax = plt.subplots(figsize=(7, 5))
if summary_plot_type == 'box':
    # Add in points to show each observation
    sns.stripplot(x='roi', y='coh_mean', hue='stim_modality',
                  data=data, color=".3", linewidth=0, dodge=True)
    # plot data summary using boxplot
    bplot = sns.boxplot(x='roi', y='coh_mean', hue='stim_modality',
                        data=data, whis=[0, 100], width=.6, 
                        palette='colorblind')
elif summary_plot_type == 'point':
    # plot data summary using pointplot
    pplot = sns.pointplot(x='roi', y='coh_mean', hue='stim_modality',
                          data=data, palette='colorblind', dodge=.4, 
                          ci=68, join=False, capsize=.05, markers='d',
                          errwidth=1.5) 
    
    # Add in points to show each observation
    sns.stripplot(x='roi', y='coh_mean', hue='stim_modality',
                  data=data, color=".3", linewidth=0, dodge=True,
                  alpha=.25)

# Tweak the visual presentation
sns.despine(trim=True, bottom=True)
plt.show()
# f.savefig('fig_aud_stimmod_x_roi.svg')

# Partial auditory coherence (true-permuted) for AV_low vs AO_high separately by ROIs

In [ ]:
# Make sure the dataframe is properly sorted
df_true = df[df['permutation'].isin(['true'])].copy().sort_values(
    by=['subID', 'coh_modality', 'stim_modality', 'acc_clarity', 'match_sample', 'roi'])
df_perm = df[df['permutation'].isin(['perm'])].copy().sort_values(
    by=['subID', 'coh_modality', 'stim_modality', 'acc_clarity', 'match_sample', 'roi'])
df_tr_p = df_true.copy().drop('permutation', axis=1)
df_tr_p['coh_mean'] = df_true['coh_mean'].to_numpy()-df_perm['coh_mean'].to_numpy()
df_tr_p['coh_max'] = df_true['coh_max'].to_numpy()-df_perm['coh_max'].to_numpy()
data = df_tr_p[df_tr_p['coh_modality'].isin(['aud']) & 
               ((df_tr_p['stim_modality'].isin(['AV']) & df_tr_p['acc_clarity'].isin(['low'])) |
                (df_tr_p['stim_modality'].isin(['AO']) & df_tr_p['acc_clarity'].isin(['high']))) &
               df_tr_p['match_sample'].isin(['none'])]
# Anova
aovrm = AnovaRM(data, 'coh_mean', 'subID', within=['stim_modality', 'roi'])
res = aovrm.fit()
print(res)

# Plot the data
sns.set_theme(style="ticks")
f, ax = plt.subplots(figsize=(7, 5))
if summary_plot_type == 'box':
    # Add in points to show each observation
    sns.stripplot(x='roi', y='coh_mean', hue='stim_modality',
                  data=data, color=".3", linewidth=0, dodge=True)
    # plot data summary using boxplot
    bplot = sns.boxplot(x='roi', y='coh_mean', hue='stim_modality',
                        data=data, whis=[0, 100], width=.6, 
                        palette='colorblind')
elif summary_plot_type == 'point':
    # plot data summary using pointplot
    pplot = sns.pointplot(x='roi', y='coh_mean', hue='stim_modality',
                          data=data, palette='colorblind', dodge=.4, 
                          ci=68, join=False, capsize=.05, markers='d',
                          errwidth=1.5) 
    
    # Add in points to show each observation
    sns.stripplot(x='roi', y='coh_mean', hue='stim_modality',
                  data=data, color=".3", linewidth=0, dodge=True,
                  alpha=.25)

# Tweak the visual presentation
sns.despine(trim=True, bottom=True)
plt.show()
# f.savefig('fig_aud_AVlow_vs_AOhigh_stimmod_x_roi.svg')

# Partial visual coherence (true-permuted) for AV vs VO separately by ROIs

In [ ]:
# Make sure the dataframe is properly sorted
df_true = df[df['permutation'].isin(['true'])].copy().sort_values(
    by=['subID', 'coh_modality', 'stim_modality', 'acc_clarity', 'match_sample', 'roi'])
df_perm = df[df['permutation'].isin(['perm'])].copy().sort_values(
    by=['subID', 'coh_modality', 'stim_modality', 'acc_clarity', 'match_sample', 'roi'])
df_tr_p = df_true.copy().drop('permutation', axis=1)
df_tr_p['coh_mean'] = df_true['coh_mean'].to_numpy()-df_perm['coh_mean'].to_numpy()
df_tr_p['coh_max'] = df_true['coh_max'].to_numpy()-df_perm['coh_max'].to_numpy()
data = df_tr_p[df_tr_p['coh_modality'].isin(['vis']) & 
               df_tr_p['stim_modality'].isin(['AV', 'VO']) & 
               df_tr_p['acc_clarity'].isin(['none']) &
               df_tr_p['match_sample'].isin(['none'])]

# Plot the data
sns.set_theme(style="ticks")
f, ax = plt.subplots(figsize=(7, 5))
if summary_plot_type == 'box':
    # Add in points to show each observation
    sns.stripplot(x='roi', y='coh_mean', hue='stim_modality',
                  data=data, color=".3", linewidth=0, dodge=True)
    # plot data summary using boxplot
    bplot = sns.boxplot(x='roi', y='coh_mean', hue='stim_modality',
                        data=data, whis=[0, 100], width=.6, 
                        palette='colorblind')
elif summary_plot_type == 'point':
    # plot data summary using pointplot
    pplot = sns.pointplot(x='roi', y='coh_mean', hue='stim_modality',
                          data=data, palette='colorblind', dodge=.4, 
                          ci=68, join=False, capsize=.05, markers='d',
                          errwidth=1.5) 
    # Add in points to show each observation
    sns.stripplot(x='roi', y='coh_mean', hue='stim_modality',
                  data=data, color=".3", linewidth=0, dodge=True,
                  alpha=.25)

# Tweak the visual presentation
sns.despine(trim=True, bottom=True)
plt.show()
# f.savefig('fig_vis_stimmod_x_roi.svg')

# Partial visual coherence (true-permuted)for AV (sample size matched to VO) vs VO separately by ROIs

In [ ]:
# Make sure the dataframe is properly sorted
df_true = df[df['permutation'].isin(['true'])].copy().sort_values(
    by=['subID', 'coh_modality', 'stim_modality', 'acc_clarity', 'match_sample', 'roi'])
df_perm = df[df['permutation'].isin(['perm'])].copy().sort_values(
    by=['subID', 'coh_modality', 'stim_modality', 'acc_clarity', 'match_sample', 'roi'])
df_tr_p = df_true.copy().drop('permutation', axis=1)
df_tr_p['coh_mean'] = df_true['coh_mean'].to_numpy()-df_perm['coh_mean'].to_numpy()
df_tr_p['coh_max'] = df_true['coh_max'].to_numpy()-df_perm['coh_max'].to_numpy()

data = df_tr_p[df_tr_p['coh_modality'].isin(['vis']) & 
               df_tr_p['acc_clarity'].isin(['none']) & 
               ((df_tr_p['stim_modality'].isin(['AV']) & df_tr_p['match_sample'].isin(['matchVO'])) |
                df_tr_p['stim_modality'].isin(['VO']))] # AV matched to VO, and VO
# Anova
aovrm = AnovaRM(data, 'coh_mean', 'subID', within=['stim_modality', 'roi'])
res = aovrm.fit()
print(res)

# Plot the data
sns.set_theme(style="ticks")
f, ax = plt.subplots(figsize=(7, 5))
if summary_plot_type == 'box':
    # Add in points to show each observation
    sns.stripplot(x='roi', y='coh_mean', hue='stim_modality',
                  data=data, color=".3", linewidth=0, dodge=True)
    # plot data summary using boxplot
    bplot = sns.boxplot(x='roi', y='coh_mean', hue='stim_modality',
                        data=data, whis=[0, 100], width=.6, 
                        palette='colorblind')
elif summary_plot_type == 'point':
    # plot data summary using pointplot
    pplot = sns.pointplot(x='roi', y='coh_mean', hue='stim_modality',
                          data=data, palette='colorblind', dodge=.4, 
                          ci=68, join=False, capsize=.05, markers='d',
                          errwidth=1.5) 
    # Add in points to show each observation
    sns.stripplot(x='roi', y='coh_mean', hue='stim_modality',
                  data=data, color=".3", linewidth=0, dodge=True,
                  alpha=.25)

# Tweak the visual presentation
sns.despine(trim=True, bottom=True)
plt.show()
# f.savefig('fig_vis_stimmod_x_roi_matchVO.svg')

# Coherence - behavioural performance correlations
**Auditory partial coherence in occipital cortex vs. word report accuracy**

In [ ]:
coh_AVlow_true = df.loc[df['stim_modality'].isin(['AV']) &
                        df['coh_modality'].isin(['aud']) & 
                        df['acc_clarity'].isin(['low']) &
                        df['permutation'].isin(['true']) &
                        df['roi'].isin(['Occ_cortex']), 'coh_mean'].to_numpy()
coh_AVlow_perm = df.loc[df['stim_modality'].isin(['AV']) &
                        df['coh_modality'].isin(['aud']) & 
                        df['acc_clarity'].isin(['low']) &
                        df['permutation'].isin(['perm']) &
                        df['roi'].isin(['Occ_cortex']), 'coh_mean'].to_numpy()
coh_AOhigh_true = df.loc[df['stim_modality'].isin(['AO']) &
                         df['coh_modality'].isin(['aud']) & 
                         df['acc_clarity'].isin(['high']) &
                         df['permutation'].isin(['true']) &
                         df['roi'].isin(['Occ_cortex']), 'coh_mean'].to_numpy()
coh_AOhigh_perm = df.loc[df['stim_modality'].isin(['AO']) &
                         df['coh_modality'].isin(['aud']) & 
                         df['acc_clarity'].isin(['high']) &
                         df['permutation'].isin(['perm']) &
                         df['roi'].isin(['Occ_cortex']), 'coh_mean'].to_numpy()
coh = (coh_AVlow_true-coh_AVlow_perm)-(coh_AOhigh_true-coh_AOhigh_perm)
acc_AVhigh = df_wr.loc[df_wr['stim_modality'].isin(['AV']) &
                       df_wr['acc_clarity'].isin(['low']), 'accuracy'].to_numpy()
acc_AOlow = df_wr.loc[df_wr['stim_modality'].isin(['AO']) &
                      df_wr['acc_clarity'].isin(['high']), 'accuracy'].to_numpy()
acc = acc_AVhigh-acc_AOlow
# Plot regression
ax = sns.regplot(x=acc, y=coh)
ax.set(xlabel='Visual preference index')
ax.set(ylabel='Auditory coherence\n(true-permuted)')
ax.set(title='Auditory partial coherence vs visual preference index\n(AVlow-AOhigh, occipital cortex)')
# Compute correlation
r, pval = stats.pearsonr(acc, coh)
print('Pearson correlation:\nr = {:.4f}\np = {:.4f}\n'.format(r, pval))

**Visual partial coherence in STG vs. word report accuracy in VO condition**

In [ ]:
coh_true = df.loc[df['stim_modality'].isin(['VO']) &
                  df['coh_modality'].isin(['vis']) & 
                  df['permutation'].isin(['true']) &
                  df['roi'].isin(['STG']), 'coh_mean'].to_numpy()
coh_perm = df.loc[df['stim_modality'].isin(['VO']) &
                  df['coh_modality'].isin(['vis']) & 
                  df['permutation'].isin(['perm']) &
                  df['roi'].isin(['STG']), 'coh_mean'].to_numpy()
# Overwriting coherence with ture-permuted coherence
coh = coh_true-coh_perm
# Appendning word report accuracies
acc = df_wr.loc[df_wr['stim_modality'].isin(['VO']), 'accuracy'].to_numpy()
# Plot data with regression line
ax = sns.regplot(x=acc, y=coh)
ax.set(xlabel='Word report accuracy (VO)')
ax.set(ylabel='Visual coherence\n(true-permuted)')
ax.set(title='Visual partial coherence vs word report accuracy\n(VO, STG)')
# Compute correlation
r, pval = stats.pearsonr(acc, coh)
print('Pearson correlation:\nr = {:.4f}\np = {:.4f}\n'.format(r, pval))

**Auditory partial coherence in right parietal cluster (AV > AO) vs. visual preference index**

In [ ]:
coh_AV_true = df_func.loc[df_func['stim_modality'].isin(['AV']) &
                          df_func['coh_modality'].isin(['aud']) & 
                          df_func['acc_clarity'].isin(['none']) &
                          df_func['permutation'].isin(['true']) &
                          df_func['roi'].isin(['aud_AV>AO']), 'coh_mean'].to_numpy()
coh_AV_perm = df_func.loc[df_func['stim_modality'].isin(['AV']) &
                          df_func['coh_modality'].isin(['aud']) & 
                          df_func['acc_clarity'].isin(['none']) &
                          df_func['permutation'].isin(['perm']) &
                          df_func['roi'].isin(['aud_AV>AO']), 'coh_mean'].to_numpy()
coh_AO_true = df_func.loc[df_func['stim_modality'].isin(['AO']) &
                          df_func['coh_modality'].isin(['aud']) & 
                          df_func['acc_clarity'].isin(['none']) &
                          df_func['permutation'].isin(['true']) &
                          df_func['roi'].isin(['aud_AV>AO']), 'coh_mean'].to_numpy()
coh_AO_perm = df_func.loc[df_func['stim_modality'].isin(['AO']) &
                          df_func['coh_modality'].isin(['aud']) & 
                          df_func['acc_clarity'].isin(['none']) &
                          df_func['permutation'].isin(['perm']) &
                          df_func['roi'].isin(['aud_AV>AO']), 'coh_mean'].to_numpy()
coh = (coh_AV_true-coh_AV_perm)-(coh_AO_true-coh_AO_perm)
acc_AVhigh = df_wr.loc[df_wr['stim_modality'].isin(['AV']) &
                       df_wr['acc_clarity'].isin(['low']), 'accuracy'].to_numpy()
acc_AOlow = df_wr.loc[df_wr['stim_modality'].isin(['AO']) &
                      df_wr['acc_clarity'].isin(['high']), 'accuracy'].to_numpy()
acc = acc_AVhigh-acc_AOlow
# Plot regression
ax = sns.regplot(x=acc, y=coh)
ax.set(xlabel='Visual preference index')
ax.set(ylabel='Auditory coherence\n(true-permuted)')
ax.set(title='Auditory partial coherence vs visual perefernce index\nRight parietal cluster AV>AO')
# Compute correlation
r, pval = stats.pearsonr(acc, coh)
print('Pearson correlation:\nr = {:.4f}\np = {:.4f}\n'.format(r, pval))

**Visual partial coherence in left anterior-temporal cluster (VO > AV) vs. lip reading performance**

In [ ]:
coh_VO_true = df_func.loc[df_func['stim_modality'].isin(['VO']) &
                          df_func['coh_modality'].isin(['vis']) & 
                          df_func['acc_clarity'].isin(['none']) &
                          df_func['permutation'].isin(['true']) &
                          df_func['roi'].isin(['vis_VO>AV']) & 
                          df_func['hemisphere'].isin(['lh']), 'coh_mean'].to_numpy()
coh_VO_perm = df_func.loc[df_func['stim_modality'].isin(['VO']) &
                          df_func['coh_modality'].isin(['vis']) & 
                          df_func['acc_clarity'].isin(['none']) &
                          df_func['permutation'].isin(['perm']) &
                          df_func['roi'].isin(['vis_VO>AV']) & 
                          df_func['hemisphere'].isin(['lh']), 'coh_mean'].to_numpy()
coh_AV_true = df_func.loc[df_func['stim_modality'].isin(['AV']) &
                          df_func['coh_modality'].isin(['vis']) & 
                          df_func['acc_clarity'].isin(['none']) &
                          df_func['permutation'].isin(['true']) &
                          df_func['roi'].isin(['vis_VO>AV']) & 
                          df_func['hemisphere'].isin(['lh']), 'coh_mean'].to_numpy()
coh_AV_perm = df_func.loc[df_func['stim_modality'].isin(['AV']) &
                          df_func['coh_modality'].isin(['vis']) & 
                          df_func['acc_clarity'].isin(['none']) &
                          df_func['permutation'].isin(['perm']) &
                          df_func['roi'].isin(['vis_VO>AV']) & 
                          df_func['hemisphere'].isin(['lh']), 'coh_mean'].to_numpy()
coh = (coh_VO_true-coh_VO_perm)-(coh_AV_true-coh_AV_perm)
# Appendning word report accuracies
acc = df_wr.loc[df_wr['stim_modality'].isin(['VO']), 'accuracy'].to_numpy()
# Plot data with regression line
ax = sns.regplot(x=acc, y=coh)
ax.set(xlabel='Word report accuracy (VO)')
ax.set(ylabel='Visual coherence\n(true-permuted)')
ax.set(title='Visual partial coherence vs lip reading performance\nLeft anterior-temporal cluster (VO>AV)')
# Compute correlation
r, pval = stats.pearsonr(acc, coh)
# We predicted the direction of this correlation, hence it is appropriate to use one-tailed p-value
pval /= 2
print('Pearson correlation:\nr = {:.4f}\np = {:.4f}\n'.format(r, pval))
# f = ax.get_figure()
# f.savefig('fig_pcoh_vis_VO-AV_vs_lipread_lh_cluster.svg')

**Visual partial coherence in right temporal cluster (VO > AV) vs. lip reading performance**

In [ ]:
coh_VO_true = df_func.loc[df_func['stim_modality'].isin(['VO']) &
                          df_func['coh_modality'].isin(['vis']) & 
                          df_func['acc_clarity'].isin(['none']) &
                          df_func['permutation'].isin(['true']) &
                          df_func['roi'].isin(['vis_VO>AV']) & 
                          df_func['hemisphere'].isin(['rh']), 'coh_mean'].to_numpy()
coh_VO_perm = df_func.loc[df_func['stim_modality'].isin(['VO']) &
                          df_func['coh_modality'].isin(['vis']) & 
                          df_func['acc_clarity'].isin(['none']) &
                          df_func['permutation'].isin(['perm']) &
                          df_func['roi'].isin(['vis_VO>AV']) & 
                          df_func['hemisphere'].isin(['rh']), 'coh_mean'].to_numpy()
coh_AV_true = df_func.loc[df_func['stim_modality'].isin(['AV']) &
                          df_func['coh_modality'].isin(['vis']) & 
                          df_func['acc_clarity'].isin(['none']) &
                          df_func['permutation'].isin(['true']) &
                          df_func['roi'].isin(['vis_VO>AV']) & 
                          df_func['hemisphere'].isin(['rh']), 'coh_mean'].to_numpy()
coh_AV_perm = df_func.loc[df_func['stim_modality'].isin(['AV']) &
                          df_func['coh_modality'].isin(['vis']) & 
                          df_func['acc_clarity'].isin(['none']) &
                          df_func['permutation'].isin(['perm']) &
                          df_func['roi'].isin(['vis_VO>AV']) & 
                          df_func['hemisphere'].isin(['rh']), 'coh_mean'].to_numpy()
coh = (coh_VO_true-coh_VO_perm)-(coh_AV_true-coh_AV_perm)
# Appendning word report accuracies
acc = df_wr.loc[df_wr['stim_modality'].isin(['VO']), 'accuracy'].to_numpy()
# Plot data with regression line
ax = sns.regplot(x=acc, y=coh)
ax.set(xlabel='Word report accuracy (VO)')
ax.set(ylabel='Visual coherence\n(true-permuted)')
ax.set(title='Visual partial coherence vs lip reading performance\nRight temporal cluster (VO>AV)')
# Compute correlation
r, pval = stats.pearsonr(acc, coh)
# We predicted the direction of this correlation, hence it is appropriate to use one-tailed p-value
pval /= 2
print('Pearson correlation:\nr = {:.4f}\np = {:.4f}\n'.format(r, pval))
# f = ax.get_figure()
# f.savefig('fig_pcoh_vis_VO-AV_vs_lipread_rh_cluster.svg')